In [ ]:
"""

MJ Price Scraper Tool v1.0


!!! Only run if using this script in Google Colab !!!

"""

!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [6]:
# List of dispensaries to scrape and preset filters to pick from

dispos = {
    "Berkshire Roots" : "https://dutchie.com/dispensary/berkshire-roots-east-boston",
    "Mission Brookline" : "https://dutchie.com/dispensary/mission-brookline",
    "NETA" : "https://dutchie.com/dispensary/neta-brookline",
    "Western Front" : "https://dutchie.com/dispensary/western-front1",
    "MedMen" : "https://dutchie.com/dispensary/medmen-fenway",
    "Rise" : "https://dutchie.com/dispensary/greenstar-herbals-inc-chelsea",
    "Happy Valley" : "https://dutchie.com/dispensary/happy-valley-east-boston",
    "Pure Oasis" : "https://dutchie.com/dispensary/pure-oasis1",
    "Ayr" : "https://dutchie.com/dispensary/ayr-dispensary-back-bay"
}

filters = {
    "Concentrate, Ascending Price" : "/products/concentrates?sortby=pricelowtohigh",
    "Flower, Ascending Price" : "/products/flower?potencycbd=0%2C50&potencythc=0%2C50&sortby=pricelowtohigh",
}

# Change this variable to pick the product you are looking for

filter = filters["Flower, Ascending Price"]


In [14]:
# Imports
import requests
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Session options
options = webdriver.ChromeOptions()
options.page_load_strategy = 'normal'
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [15]:
# Product class w/ attrs.
class Product:
  def __init__(self, name, company, type, thc, weight, price, oldprice, discount, dispo):
    self.name = name
    self.company = company
    self.type = type
    self.thc = thc
    self.weight = weight
    self.price = price
    self.oldprice = oldprice
    self.discount = discount
    self.dispo = dispo

In [16]:
# Function to scrape URL and returns the main element containing products and prod info
def scrapeURL(driver, dispo, filter): #DONE: Add filter var

    # Connect to URL
    filterURL = dispos[dispo] + filter

    # temp fix to other dispos using 3.5g vs 1/8oz as weight filter (No Longer Needed)
    # if (dispo == "MedMen" & filter.contains("Flower1_8")):
    #    filterURL = dispos[dispo] + filters["Flower3_5, Ascending Price"]


    driver.get(filterURL)
    driver.implicitly_wait(1.5)
    #time.sleep(5)

    # Age verification
    
    try:
        driver.find_element(By.CLASS_NAME, "age-confirmation-modal__StyledButton-di8wrk-0").click()
    except:
        print("")


    # Smooth scroll to scrub ALL data
    scheight = .1
    while scheight < 9.9:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/%s);" % scheight)
        scheight += .0075

    # Main element with all products
    main_element = driver.find_element(By.TAG_NAME, "main")
    
    # Searching for the product lists
    element_products = main_element.find_element(By.CLASS_NAME, "product-list__Container-sc-1arkwfu-1")
    element_div = element_products.find_element(By.TAG_NAME, "div")
    element_prod_list = element_div.find_elements(By.CLASS_NAME, "mobile-product-list-item__Container-zxgt1n-0")
    prod_list = element_prod_list[1:len(element_prod_list)-1]

    # Parsing the list into organized Product class
    products = []
    for element in prod_list:
        data = element.text.split("\n")
        if (len(data) == 7):   # Discounted Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[5], data[4], data[6], dispo)
            products.append(p)
        elif (len(data) == 6): # Staff Pick Product
            p = Product(data[1], data[2], data[3], data[3], data[4], data[5], data[5], "0%", dispo)
            products.append(p)
        elif (len(data) == 5): # Regular Product
            p = Product(data[0], data[1], data[2], data[2], data[3], data[4], data[4], "0%", dispo)
            products.append(p)
    
    print(len(products), "products scraped from", dispo)
    return products

In [17]:
products = []
i=0
for dispo in dispos: #TODO: Fix Happy Valley prices
    dispo_products = scrapeURL(driver, dispo, filter)
    products.extend(dispo_products)
    i+=1
    print("[", i, "/", len(dispos), "] Dispos scraped...")

35 products scraped from Berkshire Roots
[ 1 / 9 ] Dispos scraped...

29 products scraped from Mission Brookline
[ 2 / 9 ] Dispos scraped...

3 products scraped from NETA
[ 3 / 9 ] Dispos scraped...

20 products scraped from Western Front
[ 4 / 9 ] Dispos scraped...

19 products scraped from MedMen
[ 5 / 9 ] Dispos scraped...

11 products scraped from Rise
[ 6 / 9 ] Dispos scraped...

14 products scraped from Happy Valley
[ 7 / 9 ] Dispos scraped...

27 products scraped from Pure Oasis
[ 8 / 9 ] Dispos scraped...

90 products scraped from Ayr
[ 9 / 9 ] Dispos scraped...


In [18]:
p = products
p.sort(key=lambda x: x.price, reverse=False)

print(len(p), "products:")

for pr in p:
        print(pr.name, pr.price, pr.weight, pr.dispo)

#TODO: Add deals from dispo home blurb

#TODO: Find best deals and other info

#TODO: Clean names


#TODO: add 1g version of .5g products

#TODO: Add custom function for Sanctuary

#TODO: Categorize (Wax, Sugar, Kief, Crumble, ...)

#TODO: Data visualization (tables and graphs)

#TODO: Store logs of data w/ timestamp

248 products:
Biscotti 1g Flower $10.99 1g - Western Front
Sour Diesel 1g Flower $10.99 1g - Western Front
It's it | 14g $110.00 1/2 oz - Mission Brookline
Sugar Plum Sunset $119.99 1/2 oz - Pure Oasis
Rainbow Chip (Hybrid) - Flower, 1g $15.00 1g - NETA
Pie Face MAC (Classic) $15.00 1g - Happy Valley
Dog Patch (Select) $18.00 1g - Happy Valley
Flower Jar 1g - Jack's Peaches #18 $18.00 1g - Happy Valley
Flower Jar 1g - Jack's Peaches #2 $18.00 1g - Happy Valley
Flower Jar 1g - Jack's Peaches #3 $18.00 1g - Happy Valley
Flower Jar 1g - Jack's Peaches #7 $18.00 1g - Happy Valley
Peanut Butter Breath (Select) $18.00 1g - Happy Valley
Critical Orange Punch $199.99 1 oz - Pure Oasis
Red Mandarine $199.99 1 oz - Pure Oasis
Milk Bone $200.00 1 oz - Rise
Popcorn 1:1 Strawberry Lotus 3.5g Flower $24.99 1/8 oz - Western Front
Bubba Cheese | 3.5g $25.00 1/8 oz - Mission Brookline
Shamrock Shake | 3.5g $25.00 1/8 oz - Mission Brookline
Durban 3.5g Flower $29.99 1/8 oz - Western Front
GCC- Gelato 3.